In [30]:
from testfixtures import TempDirectory
from contextlib import contextmanager
from spectrocrunch.io import localfs,h5fs,nxfs
import spectrocrunch

@contextmanager
def tmpdir():
    dir = TempDirectory()
    try:
        yield dir.path
    except Exception as e:
        raise e
    else:
        dir.cleanup()

Directory structure
-------------------
Transparent usage of file system operations (mkdir, mv, cp, ls, open, ...) for local and hdf5 file systems:

In [41]:
def directory_structure(root):
    # Directory
    dir1 = root['dir1'].mkdir()
    assert(dir1.exists)
    assert(dir1==root['dir1'])
    
    # File
    file1 = dir1['file1.txt']
    data = 'test'
    if isinstance(root,localfs.Path):
        with file1.open(mode='w') as f:
            f.write(data)
    else:
        with file1.open(mode='w',data=data) as f:
            pass
    assert(data==file1.read())
    root['file1.txt'].link(file1)
    
    # Copy/move
    dir2 = dir1.copy(root['dir2'])
    dir3 = dir1.move(root['dir3'])
    
    # Link
    dir2lnk = root['dir2lnk'].link(dir2)
    dir3lnk = root['dir3lnk'].link(dir3)
    dir4 = dir3.move(root['dir4'])
    
    root.ls()

with tmpdir() as path:
    path = localfs.Path(path)
    directory_structure(path)
    print('')
    h5file = path['test.h5']
    path = h5fs.Path(str(h5file)+':/test')
    directory_structure(path)

+-/tmp/tmpG_Y13f
  |
  +-dir2
  | |
  | +-file1.txt
  |
  +-dir2lnk ---> dir2
  |
  +-dir3lnk -/-> dir3
  |
  +-dir4
  | |
  | +-file1.txt
  |
  +-file1.txt -/-> dir1/file1.txt

+-/tmp/tmpG_Y13f/test.h5:/test
  |
  +-dir2
  | |
  | +-file1.txt
  |
  +-dir2lnk ---> dir2
  |
  +-dir3lnk -/-> dir3
  |
  +-dir4
  | |
  | +-file1.txt
  |
  +-file1.txt -/-> dir1/file1.txt


Nexus structure
---------------
File system operations specific to the Nexus standard:

In [33]:
with tmpdir() as path:
    # Root
    root = nxfs.Path(path+'test.h5:/').nxroot()

    # NXentry
    entry = root.nxentry(name='entry0001')
    entry = root.new_nxentry()
    
    # Add data
    shape = (2,20,30)
    plotselect = entry.nxdata('plotselect')
    for signal in ['A','B','C']:
        signal = plotselect.add_signal(name=signal,dtype=float,shape=shape)
    plotselect.mark_default()
    assert(root.default==plotselect)
    assert(plotselect.signal.name=='C')
    plotselect.default_signal('B')
    assert(plotselect.signal.name=='B')
    
    # Add axes (by name or by value)
    positioners = entry.positioners()
    positioners.add_axis('y',range(shape[1]),units='um',title='vertical')
    positioners.add_axis('x',range(shape[2]),units='um',title='horizontal')
    energy = 'energy',range(5,5+shape[0]),{'units':'keV','title':'energy'}
    plotselect.set_axes(energy,'y','x')
    for ax in ['energy','y','x']:
        assert(ax in positioners)
    
    # Data processing
    entry = root.new_nxentry()
    parameters = {'a':1,'b':2}
    process,existed = entry.nxprocess(name='proc1',parameters=parameters)
    assert(process.config.read()==parameters)
    assert(existed==False)
    process,existed = entry.nxprocess(name='proc1',parameters=parameters)
    assert(existed)
    process.results['result1'].write(data=range(10))
    process.results['result2'].write(data=range(20))
    process2,existed = entry.nxprocess(name='proc2',previous=[process],parameters=parameters)
    assert(existed==False)
    assert(process2.previous_processes[0].linkdest()==process)
    
    root.ls(stats=True)

+-/tmp/tmp9ZyfhOtest.h5:/
  | @HDF5_Version = 1.10.2
  | @creator = spectrocrunch
  | @default = entry0002
  | @file_name = /tmp/tmp9ZyfhOtest.h5
  | @NX_class = NXroot
  | @file_time = 2018-11-12T15:28:52.190201
  | @h5py_version = 2.8.0
  | @file_update_time = 2018-11-12T15:28:52.875984
  |
  +-entry0001
  | | @NX_class = NXentry
  | |
  | +-end_time
  | |
  | +-start_time
  |
  +-entry0002
  | | @default = plotselect
  | | @NX_class = NXentry
  | |
  | +-end_time
  | |
  | +-instrument
  | | | @NX_class = NXinstrument
  | | |
  | | +-positioners
  | |   | @NX_class = NXcollection
  | |   |
  | |   +-energy
  | |   |   @units = keV
  | |   |   @long_name = energy
  | |   |
  | |   +-x
  | |   |   @units = um
  | |   |   @long_name = horizontal
  | |   |
  | |   +-y
  | |       @units = um
  | |       @long_name = vertical
  | |
  | +-plotselect
  | | | @signal = B
  | | | @auxiliary_signals = [u'C' u'A']
  | | | @axes = [u'energy' u'y' u'x']
  | | | @NX_class = NXdata
  | | |
  | | +